A program is nothing without data, and data in your program is primarily stored
in one of two places: the call stack and the heap. (Some values can also be
stored in CPU registers, but we won't get into that here.) All modern
programming languages use a call stack and a heap to manage memory during
execution, but they each do so a bit differently—sometimes very differently. So
in the following sections, we'll explain how Mojo helps manage memory in your
programs and how this affects the way you write code in Mojo.

## Stack and heap overview

In general, all programming languages use a call stack the same way: When a
function is called, all the execution logic and local variables with fixed-size
values are stored together and added to the top of the call stack. When another
function is called, its data is likewise added to the top of the stack. You can
think of each function's data on the call stack like a box that's placed atop
other boxes—only the contents of the top box are accessible (you cannot access
the data from a function that's lower in the stack). When a function is done,
all its data in the stack is destroyed (that memory is freed), and the function
below it is now at the top of the stack.

Notice that we said "local variables with _fixed-size_ values" are stored in
the stack. Any dynamic-sized values that can change in size at runtime are
instead stored in the heap, which is a much larger region of memory that allows
for dynamic memory access at runtime (technically, the local variable is still
stored in the call stack, but its value is a fixed-size pointer to the real
value on the heap). Additionally, any values that need to outlive the lifetime
of a function (such as an array that's passed between functions) must be stored
in the heap, because the function at the top of the call stack cannot read
values from a function lower in the stack. This sort of situation—in which
heap-allocated values are shared across functions—is where most memory errors
occur, and it's where memory management strategies vary the most between
programming languages.

Because memory is limited, it's important that programs remove unused values
from the heap ("free" the memory) as quickly as possible. Some programming
languages can free the memory automatically if the heap-allocated memory is
confined to a function on the call stack (such as a string that's used by only
one function). However, when a heap value is used in several places, the memory
management strategies vary across languages.

## Memory management strategies

Some programming languages require manual memory management. This means the
programmer must specify all heap allocations and deallocations. When done
properly, this usually makes programs execute quickly. However, the challenge
with this approach is that programmers make mistakes, especially when multiple
parts of the program need access to the same memory—it becomes difficult to
know which part of the program "owns" the data and must deallocate it. Memory
might accidentally be deallocated before the program is done with it (causing
"use-after-free" errors), or it could be deallocated twice ("double free"
errors), or the memory might never be freed ("leaked memory" errors). Mistakes
like these and others can have catastrophic results for the program, and these
bugs are often hard to track down, making it especially important that they
don't occur in the first place.

Other programming languages try to hide the complexities of memory management
from you by utilizing a "garbage collector" process that tracks all memory
usage and deallocates unused memory periodically (also known as automatic
memory management). A significant benefit of this method is that it relieves
developers from the burden of manual memory management, generally avoiding more
errors and making developers more productive. However, it incurs a performance
cost because the garbage collector interrupts the program's execution, and it
might not reclaim memory very quickly.

Mojo uses a third approach called "ownership" that relies upon a collection of
programming rules that developers must follow when passing values. The rules
ensure there is only one "owner" for each chunk of memory at a time, and that
the memory is deallocated accordingly. In this way, Mojo automatically
allocates and deallocates memory for you—when using data types with
destructors—but it does so in a way that's deterministic and safe from errors
such as use-after-free, double-free and memory leaks, and it does so with a
very low performance overhead.

Mojo's ownership model provides an excellent balance of strong memory safety and
programming productivity. It just requires that you learn some new syntax and a
few rules about how to share access to memory within your program.